In [1]:

# prompt: kaggle datasets download -d ohagwucollinspatrick/ghana-crop-disease

!kaggle datasets download -d ohagwucollinspatrick/ghana-crop-disease

Dataset URL: https://www.kaggle.com/datasets/ohagwucollinspatrick/ghana-crop-disease
License(s): Attribution 4.0 International (CC BY 4.0)
100% 10.4G/10.4G [00:55<00:00, 196MB/s]
100% 10.4G/10.4G [00:55<00:00, 201MB/s]


In [2]:
!unzip ghana-crop-disease.zip -d data


Le flux de sortie a été tronqué et ne contient que les 5000 dernières lignes.
  inflating: data/images/id_axf8fk.jpg  
  inflating: data/images/id_axfwao.jpg  
  inflating: data/images/id_axjera.jpg  
  inflating: data/images/id_axktj2.jpg  
  inflating: data/images/id_axojpi.jpg  
  inflating: data/images/id_ay7lil.jpg  
  inflating: data/images/id_ay85js.jpg  
  inflating: data/images/id_ayd2my.jpg  
  inflating: data/images/id_ayeiia.jpg  
  inflating: data/images/id_ayklsb.jpg  
  inflating: data/images/id_azbboc.jpg  
  inflating: data/images/id_azkhqh.jpg  
  inflating: data/images/id_azriv5.jpg  
  inflating: data/images/id_azzp7g.jpg  
  inflating: data/images/id_b11ovu.jpg  
  inflating: data/images/id_b15b2l.jpg  
  inflating: data/images/id_b1awql.jpg  
  inflating: data/images/id_b1e9sg.jpg  
  inflating: data/images/id_b1mlcd.jpg  
  inflating: data/images/id_b1sv9d.jpg  
  inflating: data/images/id_b2do8m.jpg  
  inflating: data/images/id_b2e92m.jpg  
  inflating: data/im

In [3]:
import pandas as pd
df = pd.read_csv('/content/data/Train.csv')
unique_classes = df['class'].unique()
class_mapping = {cls: idx for idx, cls in enumerate(unique_classes)}
print(class_mapping)


{'Pepper_Bacterial_Spot': 0, 'Pepper_Fusarium': 1, 'Corn_Cercospora_Leaf_Spot': 2, 'Corn_Common_Rust': 3, 'Tomato_Early_Blight': 4, 'Pepper_Septoria': 5, 'Tomato_Septoria': 6, 'Pepper_Leaf_Curl': 7, 'Pepper_Leaf_Mosaic': 8, 'Corn_Streak': 9, 'Corn_Healthy': 10, 'Pepper_Healthy': 11, 'Tomato_Healthy': 12, 'Pepper_Late_Blight': 13, 'Tomato_Late_Blight': 14, 'Pepper_Cercospora': 15, 'Tomato_Fusarium': 16, 'Pepper_Leaf_Blight': 17, 'Tomato_Leaf_Curl': 18, 'Tomato_Bacterial_Spot': 19, 'Tomato_Mosaic': 20, 'Pepper_Early_Blight': 21, 'Corn_Northern_Leaf_Blight': 22}


In [4]:
# import nbformat

# # Path to the notebook
# notebook_path = '/kaggle/input/ghana-csv-data-starter-notebook/Rail_Challenge_Starter.ipynb'  # Update this with the correct path

# # Load the notebook
# with open(notebook_path, 'r') as f:
#     notebook_content = nbformat.read(f, as_version=4)

# # Extract code cells
# code_cells = [cell['source'] for cell in notebook_content.cells if cell.cell_type == 'code']

# # Print the code cells
# for i, code in enumerate(code_cells):
#     print(f"Code Cell {i + 1}:\n{code}\n")


In [5]:
# Import libraries
!pip install ultralytics
import pandas as pd
import os
from pathlib import Path
import shutil
from sklearn.model_selection import train_test_split
from tqdm.notebook import tqdm
import cv2
import yaml
import matplotlib.pyplot as plt
from ultralytics import YOLO
import multiprocessing



# Set the data directory
DATA_DIR = Path('/content/data')

# Load train and test files
train = pd.read_csv(DATA_DIR / 'Train.csv')
test = pd.read_csv(DATA_DIR / 'Test.csv')
ss = pd.read_csv(DATA_DIR / 'SampleSubmission.csv')

# Add an image_path column
train['image_path'] = [Path('/content/data/images/' + x) for x in train.Image_ID]
test['image_path'] = [Path('/content/data/images/' + x) for x in test.Image_ID]

# Map string classes to integers (label encoding targets)
train['class_id'] = train['class'].map(
    {'Pepper_Bacterial_Spot': 0, 'Pepper_Fusarium': 1, 'Corn_Cercospora_Leaf_Spot': 2, 'Corn_Common_Rust': 3, 'Tomato_Early_Blight': 4, 'Pepper_Septoria': 5, 'Tomato_Septoria': 6, 'Pepper_Leaf_Curl': 7, 'Pepper_Leaf_Mosaic': 8, 'Corn_Streak': 9, 'Corn_Healthy': 10, 'Pepper_Healthy': 11, 'Tomato_Healthy': 12, 'Pepper_Late_Blight': 13, 'Tomato_Late_Blight': 14, 'Pepper_Cercospora': 15, 'Tomato_Fusarium': 16, 'Pepper_Leaf_Blight': 17, 'Tomato_Leaf_Curl': 18, 'Tomato_Bacterial_Spot': 19, 'Tomato_Mosaic': 20, 'Pepper_Early_Blight': 21, 'Corn_Northern_Leaf_Blight': 22})

# Split data into training and validation
train_unique_imgs_df = train.drop_duplicates(subset=['Image_ID'], ignore_index=True)
X_train, X_val = train_test_split(train_unique_imgs_df, test_size=0.25, stratify=train_unique_imgs_df['class'], random_state=42)

X_train = train[train.Image_ID.isin(X_train.Image_ID)]
X_val = train[train.Image_ID.isin(X_val.Image_ID)]

# Check the shapes of training and validation data
print(X_train.shape, X_val.shape)

# Define directories for images and labels
TRAIN_IMAGES_DIR = Path('/content/data1/train/images')
VAL_IMAGES_DIR = Path('/content/data1/val/images')
TEST_IMAGES_DIR = Path('/content/data1/test/images')
TRAIN_LABELS_DIR = Path('/content/data1/train/labels')
VAL_LABELS_DIR = Path('/content/data1/val/labels')
TEST_LABELS_DIR = Path('/content/data1/test/labels')

# Create necessary directories
for DIR in [TRAIN_IMAGES_DIR, VAL_IMAGES_DIR, TEST_IMAGES_DIR, TRAIN_LABELS_DIR, VAL_LABELS_DIR, TEST_LABELS_DIR]:
    if DIR.exists():
        shutil.rmtree(DIR)
    DIR.mkdir(parents=True, exist_ok=True)

# Copy train, val, and test images to their respective dirs
for img in tqdm(X_train.image_path.unique()):
    shutil.copy(img, TRAIN_IMAGES_DIR / img.parts[-1])

for img in tqdm(X_val.image_path.unique()):
    shutil.copy(img, VAL_IMAGES_DIR / img.parts[-1])

for img in tqdm(test.image_path.unique()):
    shutil.copy(img, TEST_IMAGES_DIR / img.parts[-1])

# Function to convert the bounding boxes to YOLO format and save them
def save_yolo_annotation(row):
    image_path, class_id, output_dir = row['image_path'], row['class_id'], row['output_dir']

    img = cv2.imread(str(image_path))
    if img is None:
        raise ValueError(f"Could not read image from path: {image_path}")

    height, width, _ = img.shape
    label_file = Path(output_dir) / f"{Path(image_path).stem}.txt"

    ymin, xmin, ymax, xmax = row['ymin'], row['xmin'], row['ymax'], row['xmax']

    # Normalize the coordinates
    x_center = (xmin + xmax) / 2 / width
    y_center = (ymin + ymax) / 2 / height
    bbox_width = (xmax - xmin) / width
    bbox_height = (ymax - ymin) / height

    with open(label_file, 'a') as f:
        f.write(f"{class_id} {x_center:.6f} {y_center:.6f} {bbox_width:.6f} {bbox_height:.6f}\n")

# Parallelize the annotation saving process
def process_dataset(dataframe, output_dir):
    dataframe['output_dir'] = output_dir
    with multiprocessing.Pool() as pool:
        list(tqdm(pool.imap(save_yolo_annotation, dataframe.to_dict('records')), total=len(dataframe)))

# Save train and validation labels to their respective dirs
process_dataset(X_train, TRAIN_LABELS_DIR)
process_dataset(X_val, VAL_LABELS_DIR)

# Create a data.yaml file required by YOLO
class_names = train['class'].unique().tolist()
num_classes = len(class_names)

data_yaml = {
    'train': str(TRAIN_IMAGES_DIR),
    'val': str(VAL_IMAGES_DIR),
    'nc': num_classes,
    'names': class_names
}

# Save the data.yaml file
yaml_path = Path('data.yaml')
with open(yaml_path, 'w') as file:
    yaml.dump(data_yaml, file, default_flow_style=False)

# Load a YOLO pretrained model
model = YOLO('yolov8s.pt')


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 879.0/879.0 kB 13.9 MB/s eta 0:00:00
Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
(30777, 9) (10252, 9)


  0%|          | 0/3676 [00:00<?, ?it/s]

  0%|          | 0/1226 [00:00<?, ?it/s]

  0%|          | 0/2101 [00:00<?, ?it/s]

<ipython-input-5-25d68a27a623>:91: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataframe['output_dir'] = output_dir


  0%|          | 0/30777 [00:00<?, ?it/s]

<ipython-input-5-25d68a27a623>:91: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataframe['output_dir'] = output_dir


  0%|          | 0/10252 [00:00<?, ?it/s]

100%|██████████| 21.5M/21.5M [00:00<00:00, 138MB/s] 


In [7]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


'/content/drive/MyDrive/Data Science Cours , Projets Personnels/Zindi/Ghana Crop Disease Detection Challenge/test_submission.jpg'

In [10]:
model.train(
    data='data.yaml',          # Path to the dataset configuration
    epochs=10,                 # Number of epochs
    imgsz=540,               # Image size
    batch=16,                  # Batch size
    device='cpu',               # Use the first GPU (0 for the first GPU)
    patience=3                # Number of epochs with no improvement after which training will stop
)

# Validate the model on the validation seta
model.val()

# Make predictions on test images and save them to a CSV file
image_files = os.listdir(TEST_IMAGES_DIR)
all_data = []

for image_file in tqdm(image_files):
    img_path = os.path.join(TEST_IMAGES_DIR, image_file)
    results = model(img_path)

    boxes = results[0].boxes.xyxy.tolist()
    classes = results[0].boxes.cls.tolist()
    confidences = results[0].boxes.conf.tolist()
    names = results[0].names

    incorrect_prediction = 0

    if not boxes:
        incorrect_prediction = incorrect_prediction +   1
        all_data.append({
            'Image_ID': image_file,
            'class': 'NEG',
            'confidence': 1.0,
            'ymin': 0,
            'xmin': 0,
            'ymax': 0,
            'xmax': 0
        })
    else:
        for box, cls, conf in zip(boxes, classes, confidences):
            x1, y1, x2, y2 = box
            detected_class = names[int(cls)]

            all_data.append({
                'Image_ID': image_file,
                'class': detected_class,
                'confidence': conf,
                'ymin': y1,
                'xmin': x1,
                'ymax': y2,
                'xmax': x2
            })
print(f'========== Total Incorrect predction or dummy prediction {incorrect_prediction}================')
# Convert the results to a DataFrame and save it
sub = pd.DataFrame(all_data)
sub.to_csv('/content/data1/benchmark_submission.csv', index=False)

from google.colab import drive
import shutil


# Step 2: Copy the submission file to Google Drive
shutil.copy('/content/data1/benchmark_submission.csv', '/content/drive/MyDrive/Data Science Cours , Projets Personnels/Zindi/Ghana Crop Disease Detection Challenge/benchmark_submission.csv')



Ultralytics 8.3.27 🚀 Python-3.10.12 torch-2.5.0+cpu CPU (Intel Xeon 2.00GHz)
engine/trainer: task=detect, mode=train, model=yolov8s.pt, data=data.yaml, epochs=10, time=None, patience=3, batch=16, imgsz=540, save=True, save_period=-1, cache=False, device=cpu, workers=8, project=None, name=train3, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, show_boxes=True, line_width=None, 

train: Scanning /content/data1/train/labels.cache... 3676 images, 0 backgrounds, 0 corrupt: 100%|██████████| 3676/3676 [00:00<?, ?it/s]
val: Scanning /content/data1/val/labels.cache... 1226 images, 0 backgrounds, 0 corrupt: 100%|██████████| 1226/1226 [00:00<?, ?it/s]

Plotting labels to runs/detect/train3/labels.jpg... 


optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.00037, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
TensorBoard: model graph visualization added ✅
Image sizes 544 train, 544 val
Using 0 dataloader workers
Logging results to runs/detect/train3
Starting training for 10 epochs...
Closing dataloader mosaic

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


       1/10         0G      2.663      4.468      1.908        208        544: 100%|██████████| 230/230 [14:43<00:00,  3.84s/it]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 39/39 [02:25<00:00,  3.73s/it]


...

val: Scanning /content/data1/val/labels.cache... 1226 images, 0 backgrounds, 0 corrupt: 100%|██████████| 1226/1226 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 77/77 [01:58<00:00,  1.54s/it]


                   all       1226      10252      0.382      0.294      0.232      0.101
 Pepper_Bacterial_Spot        132        476      0.333      0.178      0.158     0.0423
       Pepper_Fusarium         52        123      0.405      0.553      0.454      0.157
Corn_Cercospora_Leaf_Spot        202       1600      0.416      0.401      0.365       0.14
      Corn_Common_Rust         79        442      0.309      0.385      0.276      0.115
   Tomato_Early_Blight         56        476      0.325      0.313      0.278      0.125
       Pepper_Septoria         40        697      0.462     0.0111     0.0992     0.0347
       Tomato_Septoria        119       1588      0.325      0.226      0.192     0.0635
      Pepper_Leaf_Curl        138        374      0.301      0.227      0.173     0.0581
    Pepper_Leaf_Mosaic        147        523          1          0     0.0242    0.00632
           Corn_Streak         87        804      0.267       0.51      0.271     0.0895
          Corn_Hea

  0%|          | 0/2101 [00:00<?, ?it/s]

Le flux de sortie a été tronqué et ne contient que les 5000 dernières lignes.
Speed: 1.9ms preprocess, 53.3ms inference, 0.8ms postprocess per image at shape (1, 3, 256, 544)

image 1/1 /content/data1/test/images/id_k6x9x7.jpg: 384x544 9 Corn_Streaks, 66.1ms
Speed: 1.9ms preprocess, 66.1ms inference, 0.9ms postprocess per image at shape (1, 3, 384, 544)

image 1/1 /content/data1/test/images/id_i7sqx3.jpg: 256x544 7 Corn_Common_Rusts, 53.4ms
Speed: 1.8ms preprocess, 53.4ms inference, 1.0ms postprocess per image at shape (1, 3, 256, 544)

image 1/1 /content/data1/test/images/id_irh248.jpg: 416x544 29 Tomato_Late_Blights, 69.4ms
Speed: 2.8ms preprocess, 69.4ms inference, 1.0ms postprocess per image at shape (1, 3, 416, 544)

image 1/1 /content/data1/test/images/id_pnfs7v.jpg: 416x544 5 Tomato_Septorias, 70.5ms
Speed: 2.9ms preprocess, 70.5ms inference, 0.9ms postprocess per image at shape (1, 3, 416, 544)

image 1/1 /content/data1/test/images/id_h2x7of.jpg: 416x544 2 Pepper_Bacterial_Spot

'/content/drive/MyDrive/Data Science Cours , Projets Personnels/Zindi/Ghana Crop Disease Detection Challenge/benchmark_submission.csv'

In [13]:
# prompt: model performance

from ultralytics import YOLO

# ... (Your existing code) ...

# After model training and validation

# Get the validation metrics
metrics = model.val()

# Access specific metrics (e.g., mAP, precision, recall)
map50 = metrics.box.map50
map75 = metrics.box.map75
precision = metrics.box.p
recall = metrics.box.r
f1 = metrics.box.f1

print(f"mAP@0.5: {map50}")
print(f"mAP@0.75: {map75}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1-score: {f1}")

# You can also plot the metrics using the model.plot_metrics() function
# model.plot_metrics()

# You can further analyze the model performance by looking at the confusion matrix,
# precision-recall curve, and other relevant metrics.

# ... (Rest of your code) ...

Ultralytics 8.3.27 🚀 Python-3.10.12 torch-2.5.0+cpu CPU (Intel Xeon 2.00GHz)


val: Scanning /content/data1/val/labels.cache... 1226 images, 0 backgrounds, 0 corrupt: 100%|██████████| 1226/1226 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 77/77 [01:53<00:00,  1.48s/it]


                   all       1226      10252      0.382      0.294      0.232      0.101
 Pepper_Bacterial_Spot        132        476      0.333      0.178      0.158     0.0423
       Pepper_Fusarium         52        123      0.405      0.553      0.454      0.157
Corn_Cercospora_Leaf_Spot        202       1600      0.416      0.401      0.365       0.14
      Corn_Common_Rust         79        442      0.309      0.385      0.276      0.115
   Tomato_Early_Blight         56        476      0.325      0.313      0.278      0.125
       Pepper_Septoria         40        697      0.462     0.0111     0.0992     0.0347
       Tomato_Septoria        119       1588      0.325      0.226      0.192     0.0635
      Pepper_Leaf_Curl        138        374      0.301      0.227      0.173     0.0581
    Pepper_Leaf_Mosaic        147        523          1          0     0.0242    0.00632
           Corn_Streak         87        804      0.267       0.51      0.271     0.0895
          Corn_Hea

In [ ]:
#apikey for model acessing
# 928e0393be6ece8cbbd9a973386809288cb48eac

"""
Discussion Zindi ::
Stater Notebook Ghana Crop Disease Detection
I have used used   model = YOLO('yolov8m.pt')   pretrained model for refinding on my data sets I have ajust the basic code to refined this model and make submission file. You have to input api key you can get  from this website https://wandb.ai/site/  singup or login if have account  after copy  apikey from   which is used required in accessing pretrained model of YOLO While runing the code.       Here is link  of  Notebook on kaggle ()
"""